In [2]:
# This is nearly unedited code, whose main functions are featured in VirtualFluxConstruction in a cleaner,
# more user/reader friendly version. These functions were used throughout the code whenever we needed to calculate
# coefficients for virtual fluxes and their normalizations.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import uproot4 as uproot
import scipy.stats
from sklearn import linear_model
from scipy import optimize
import warnings; warnings.simplefilter('ignore')

def GetCoefficients(loc,scale,alpha=1,nonzero=58,model=None,years=5,rescaling_const=None):
    # Get DUNE near detector muon neutrino flux as a function of off-axis angle
    with uproot.open("Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians
    #
    # From oa_flux in [neutrinos cm^-2 per POT per year] to oa_events in [neutrinos per GeV per Year]
    #cross_section = 10e-38*np.array([1.7564017,1.7524341,1.7496688,1.7444551,1.7370047,1.7231702,1.7069778,1.6844808,1.6565275,1.6218245,1.5793541,1.5299167,1.4797076,1.4227808,1.3687412,1.3173940,1.2662984,1.2205544,1.1756606,1.1365877,1.1000308,1.0661938,1.0359266,1.0081116,0.98149143,0.95945773,0.93715417,0.91382316,0.89369118,0.87390051,0.85512638,0.83607184,0.81882318,0.80275432,0.78789418,0.76993262,0.75490589,0.74114020,0.72626038,0.71124357,0.69732170,0.68273972,0.66883140,0.65590717,0.64289072,0.62991282,0.61805200,0.60649662,0.59396039,0.58324116,0.57177118,0.56149453,0.55099552,0.54106390,0.53076420,0.52044680,0.51228947]) #[cm^2 nucleon^-1]
    xsecs_string = open("../xsecs.txt", "r").read().split("\n")
    cross_section = 1e-38*np.array([float(x) for x in xsecs_string])
    N_target = 1.435e30 #[nucleon]
    E = 3.62e19 #[POT per year]
    epsilon = 1
    cross_section = np.tile(cross_section, (oa_flux.shape[0],1))
    oa_events = oa_flux * cross_section * epsilon * E * N_target * 1e-2 * years
    events_vector = np.array([oa_events[:,k].sum()/100 for k in range(oa_events.shape[1])])
    events_vector = open("norms.txt", "r").read().split("\n")[:-1]
    events_vector = [float(n) for n in normalization]
    #
    target_loc = loc
    target_scale = scale #change this back to 0.1
    rescaling = oa_events.max()
    #rescaling = 1
    if rescaling_const != None:
        rescaling = rescaling_const
    energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:])/2.
    target_flux = [scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale)*rescaling for x in energy_bin_centers] #here we should scale the gaussian
    #
    if (model == None):
        x , residuals, rank, s = np.linalg.lstsq(oa_events, target_flux, rcond=None)
        print("none")
    else:
        if (model == linear_model.OrthogonalMatchingPursuit):
            lm = model(n_nonzero_coefs=nonzero,tol=alpha,fit_intercept=False)
        else:
            lm = model(alpha=alpha,fit_intercept=False)
        lm.fit(oa_events,target_flux)
        x = lm.coef_
    return list(x)

def GetCoefficientsFlux(loc,scale,alpha,nonzero=58,model=None,years=5,rescaling_const=None):
    # Get DUNE near detector muon neutrino flux as a function of off-axis angle
    with uproot.open("Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians
    #
    # From oa_flux in [neutrinos cm^-2 per POT per year] to oa_events in [neutrinos per GeV per Year]
    #cross_section = 10e-38*np.array([1.7564017,1.7524341,1.7496688,1.7444551,1.7370047,1.7231702,1.7069778,1.6844808,1.6565275,1.6218245,1.5793541,1.5299167,1.4797076,1.4227808,1.3687412,1.3173940,1.2662984,1.2205544,1.1756606,1.1365877,1.1000308,1.0661938,1.0359266,1.0081116,0.98149143,0.95945773,0.93715417,0.91382316,0.89369118,0.87390051,0.85512638,0.83607184,0.81882318,0.80275432,0.78789418,0.76993262,0.75490589,0.74114020,0.72626038,0.71124357,0.69732170,0.68273972,0.66883140,0.65590717,0.64289072,0.62991282,0.61805200,0.60649662,0.59396039,0.58324116,0.57177118,0.56149453,0.55099552,0.54106390,0.53076420,0.52044680,0.51228947]) #[cm^2 nucleon^-1]
    #
    target_loc = loc
    target_scale = scale #change this back to 0.1
    rescaling = oa_flux.max()
    #rescaling = 1
    if rescaling_const != None:
        rescaling = rescaling_const
    energy_bin_centers = np.add(energy_bins[:-1], energy_bins[1:])/2.
    target_flux = [scipy.stats.norm.pdf(x, loc = target_loc, scale = target_scale)*rescaling for x in energy_bin_centers] #here we should scale the gaussian
    #
    if (model == None):
        x , residuals, rank, s = np.linalg.lstsq(oa_flux, target_flux, rcond=None)
        print("none")
    else:
        if (model == linear_model.OrthogonalMatchingPursuit):
            lm = model(n_nonzero_coefs=nonzero,tol=alpha,fit_intercept=False)
        else:
            lm = model(alpha=alpha,fit_intercept=False)
        lm.fit(oa_flux,target_flux)
        x = lm.coef_
    print("Gaussian width: " + str(get_gaussian_std(lm.predict(oa_flux),energy_bin_centers,oa_flux)))
    return list(x), get_gaussian_std(lm.predict(oa_flux),energy_bin_centers,oa_flux)

def get_gaussian_std(prediction,energy_bin_centers,oa_events):
    gfit = optimize.curve_fit(gaussian,energy_bin_centers,prediction)[0]
    loc,scale = gfit[0],gfit[1]
    return scale

def gaussian(x,loc,scale):
    with uproot.open("Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians
    rescaling = oa_flux.max()
    return (1/(scale*((2*np.pi)**0.5)))*np.exp((-((x-loc)/scale)**2)/2)*rescaling

def get_normalization(coeffs):
    with uproot.open("Fluxes.ND.root") as fFluxes :
        oa_flux = fFluxes['LBNF_numu_flux'].values()
        energy_bins = fFluxes['LBNF_numu_flux'].axis(0).edges() # in GeV
        angle_bins = fFluxes['LBNF_numu_flux'].axis(1).edges() # in milliradians
    oa_flux = oa_flux.transpose()
    first_bin_flux = 8.9769702e-08
    norm = (oa_flux[0].sum()/100)/first_bin_flux
    oa_flux = oa_flux/norm
    print(oa_flux[0].sum()/100)
    oa_flux = oa_flux.transpose()
    lin_combo = np.matmul(oa_flux, coeffs)*0.01
    return lin_combo.sum()